# Анализ таблиц JDWH - Прод vs Тест

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 200)
#pd.set_option('display.max_rows', 500)

## Часть III. Индексы.

## 1. Загрузка и первичное преобразование данных

### Прод

In [3]:
prod = pd.read_csv('./data/indexes_prod_14122020.csv')

In [4]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"


Добавим поле full_table_name

In [5]:
prod['full_table_name'] = prod['schemaname'] + '.' + prod['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [6]:
prod['uniq'] = prod.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
prod['short_idx_def'] = prod.indexdef.apply(lambda x: x.split('USING')[1].strip())
prod['idx_type'] = prod.short_idx_def.apply(lambda x: x.split('(')[0].strip())
prod['ind_fields'] = prod.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())#x.split('(', 1)[1]

In [7]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)",dwh_bckp.amara_corr_overdues,1,"btree (id, insert_date)",btree,"id, insert_date"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch,1,"btree (id, insert_date)",btree,"id, insert_date"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch_forecast,1,"btree (id, insert_date)",btree,"id, insert_date"


Убираем партиции

In [8]:
prod = prod[prod['schemaname'] + '.' + prod['tablename'] == prod['schemaname'] + '.' + prod['base_name']]
prod.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
3881,dwh_dds,ref_product,ref_product,ref_prd_prd_gid_idx,2,1,"CREATE INDEX ref_prd_prd_gid_idx ON dwh_dds.ref_product USING btree (prd_gid, ""prd$start_date"", ""prd$end_date"" DESC, ""prd$row_status"") WHERE ((""prd$row_status"")::text = 'A'::text)",dwh_dds.ref_product,0,"btree (prd_gid, ""prd$start_date"", ""prd$end_date"" DESC, ""prd$row_status"") WHERE ((""prd$row_status"")::text = 'A'::text)",btree,"prd_gid, ""prd$start_date"", ""prd$end_date"" DESC, ""prd$row_status"""
5299,dwh_stage,s01_g_clirole,s01_g_clirole,s01_g_clirole_id_idx,2,1,"CREATE INDEX s01_g_clirole_id_idx ON dwh_stage.s01_g_clirole USING btree (id, status)",dwh_stage.s01_g_clirole,0,"btree (id, status)",btree,"id, status"
3722,dwh_dds,dict_cont_type,dict_cont_type,dict_cont_type_pk,1,1,CREATE UNIQUE INDEX dict_cont_type_pk ON dwh_dds.dict_cont_type USING btree (conttp_cd),dwh_dds.dict_cont_type,1,btree (conttp_cd),btree,conttp_cd


==============================================================================================================

### Тест

In [9]:
test = pd.read_csv('./data/indexes_test_14122020.csv')

In [10]:
test.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_short,amara_rating_fitch_short,pk_bckp_amara_rating_fitch_short,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_short ON dwh_bckp.amara_rating_fitch_short USING btree (id, insert_date)"


Добавим поле full_table_name

In [11]:
test['full_table_name'] = test['schemaname'] + '.' + test['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [12]:
test['uniq'] = test.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
test['short_idx_def'] = test.indexdef.apply(lambda x: x.split('USING')[1].strip())
test['idx_type'] = test.short_idx_def.apply(lambda x: x.split('(')[0].strip())
test['ind_fields'] = test.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [13]:
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
8820,dwh_stage,s01_t_trndtl,s01_t_trndtl_1_prt_160_2021_04_22,s01_t_trndtl_1_prt_160_2021_04_22_id_idx,4,4,CREATE INDEX s01_t_trndtl_1_prt_160_2021_04_22_id_idx ON dwh_stage.s01_t_trndtl_1_prt_160_2021_04_22 USING btree (id),dwh_stage.s01_t_trndtl_1_prt_160_2021_04_22,0,btree (id),btree,id
1727,dwh_bckp,s01_t_dea_bkp,s01_t_dea_bkp_1_prt_208_2020_09_13,s01_t_dea_bkp_1_prt_208_2020_09_13_dea_id_idx,2,2,CREATE INDEX s01_t_dea_bkp_1_prt_208_2020_09_13_dea_id_idx ON dwh_bckp.s01_t_dea_bkp_1_prt_208_2020_09_13 USING btree (dea_id),dwh_bckp.s01_t_dea_bkp_1_prt_208_2020_09_13,0,btree (dea_id),btree,dea_id
2816,dwh_dds,acc_transaction,acc_transaction_1_prt_158_2020_04_16,acc_transaction_1_prt_158_2020_04_16_trn_dt_acnt_gid_idx,4,2,CREATE INDEX acc_transaction_1_prt_158_2020_04_16_trn_dt_acnt_gid_idx ON dwh_dds.acc_transaction_1_prt_158_2020_04_16 USING btree (trn_dt_acnt_gid),dwh_dds.acc_transaction_1_prt_158_2020_04_16,0,btree (trn_dt_acnt_gid),btree,trn_dt_acnt_gid


Убираем партиции

In [14]:
test = test[test['schemaname'] + '.' + test['tablename'] == test['schemaname'] + '.' + test['base_name']]
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
6615,dwh_stage,s01_g_bnkdsc_std,s01_g_bnkdsc_std,s01_g_bnkdsc_std_pk,1,1,CREATE UNIQUE INDEX s01_g_bnkdsc_std_pk ON dwh_stage.s01_g_bnkdsc_std USING btree (id),dwh_stage.s01_g_bnkdsc_std,1,btree (id),btree,id
8223,dwh_stage,s01_t_deaext,s01_t_deaext,pk_t_deaext_refer,1,1,CREATE UNIQUE INDEX pk_t_deaext_refer ON dwh_stage.s01_t_deaext USING btree (refer),dwh_stage.s01_t_deaext,1,btree (refer),btree,refer
4184,dwh_dds,ref_pledge_obj_link,ref_pledge_obj_link,ref_pledge_obj_link_tmp_v2_pledobjlnk_crncy_cd_idx,3,2,CREATE INDEX ref_pledge_obj_link_tmp_v2_pledobjlnk_crncy_cd_idx ON dwh_dds.ref_pledge_obj_link USING btree (pledobjlnk_dlpled_gid),dwh_dds.ref_pledge_obj_link,0,btree (pledobjlnk_dlpled_gid),btree,pledobjlnk_dlpled_gid


In [15]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 0 to 7959
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       581 non-null    object
 1   base_name        581 non-null    object
 2   tablename        581 non-null    object
 3   indexname        581 non-null    object
 4   inds_count       581 non-null    int64 
 5   ind              581 non-null    int64 
 6   indexdef         581 non-null    object
 7   full_table_name  581 non-null    object
 8   uniq             581 non-null    int64 
 9   short_idx_def    581 non-null    object
 10  idx_type         581 non-null    object
 11  ind_fields       581 non-null    object
dtypes: int64(3), object(9)
memory usage: 59.0+ KB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 11540
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       594 non-null    object
 1   base_name        594 non-null    object
 2   tablename        594 non-null    object
 3   indexname        594 non-null    object
 4   inds_count       594 non-null    int64 
 5   ind              594 non-null    int64 
 6   indexdef         594 non-null    object
 7   full_table_name  594 non-null    object
 8   uniq             594 non-null    int64 
 9   short_idx_def    594 non-null    object
 10  idx_type         594 non-null    object
 11  ind_fields       594 non-null    object
dtypes: int64(3), object(9)
memory usage: 60.3+ KB


### Сверяем только те таблицы, что есть и на Прод и на Тест

Удалим лишние столбцы

Прод

In [17]:
#Проверим сначала частный случай, когда столбец ind_fields содержит пустоту
prod[prod.ind_fields.str.strip()=='']

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
14,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_pk2_idx,4,3,CREATE INDEX s01_led_acc_det_bkp_pk2_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (((pk2)::integer)),dwh_bckp.s01_led_acc_det_bkp,0,btree (((pk2)::integer)),btree,
5346,dwh_stage,s01_ledacc_det,s01_ledacc_det,s01_ledacc_det_pk2_idx,4,4,CREATE INDEX s01_ledacc_det_pk2_idx ON dwh_stage.s01_ledacc_det USING btree (((pk2)::integer)),dwh_stage.s01_ledacc_det,0,btree (((pk2)::integer)),btree,


Эти таблицы можно проверить вручную. Создаем список таблиц для сравнения вручную

In [18]:
tabs_nonstandard_indexdef = ['dwh_stage.s01_led_acc_det_bkp', 'dwh_stage.s01_ledacc_det']

In [19]:
#Удаляем лишние столбцы
prod.drop(['base_name', 'indexdef', 'short_idx_def'], inplace=True, axis = 1)
prod.sample(3)

,schemaname,tablename,indexname,inds_count,ind,full_table_name,uniq,idx_type,ind_fields
3750,dwh_dds,dict_sources,dict_sources_pk,1,1,dwh_dds.dict_sources,1,btree,src_cd
5266,dwh_stage,s01_g_accbln,s01_g_accbln_dep_id_idx,4,3,dwh_stage.s01_g_accbln,0,btree,"dep_id, ord_id"
5391,dwh_stage,s01_sk_issuequotes,pk_s01_sk_issuequotes,1,1,dwh_stage.s01_sk_issuequotes,1,btree,"id, fromdate, infagn_id, type_quotes"


Test

In [20]:
#Проверим сначала частный случай, когда столбец ind_fields содержит пустоту
test[test.ind_fields.str.strip()=='']

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
16,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_pk2_idx,4,3,CREATE INDEX s01_led_acc_det_bkp_pk2_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (((pk2)::integer)),dwh_bckp.s01_led_acc_det_bkp,0,btree (((pk2)::integer)),btree,
6687,dwh_stage,s01_ledacc_det,s01_ledacc_det,s01_ledacc_det_pk2_idx,4,4,CREATE INDEX s01_ledacc_det_pk2_idx ON dwh_stage.s01_ledacc_det USING btree (((pk2)::integer)),dwh_stage.s01_ledacc_det,0,btree (((pk2)::integer)),btree,


Эти таблицы можно проверить вручную

In [21]:
#Удаляем лишние столбцы
test.drop(['base_name', 'indexdef', 'short_idx_def'], inplace=True, axis = 1)
test.sample(3)

,schemaname,tablename,indexname,inds_count,ind,full_table_name,uniq,idx_type,ind_fields
6530,dwh_dm,dm_z12,dm_z12_dm_z12$report_date_IDX,1,1,dwh_dm.dm_z12,0,btree,"""dm_z12$report_date"" DESC"
6562,dwh_polygon,geo,geo_pkey,1,1,dwh_polygon.geo,1,btree,id
4098,dwh_dds,ref_client_priv,ref_client_priv_dprt_gid_idx,2,1,dwh_dds.ref_client_priv,0,btree,clprv_dprt_gid


### Выравниваем Прод и Тест

In [22]:
prod2 = prod[['full_table_name']]
prod2.head(2)

,full_table_name
0,dwh_bckp.amara_corr_overdues
1,dwh_bckp.amara_rating_fitch


In [23]:
test2 = test[['full_table_name']]
test2.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


In [24]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 11540
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   full_table_name  594 non-null    object
dtypes: object(1)
memory usage: 9.3+ KB


Готовим список нужных имен таблиц (которые есть и на Прод и на Тест)

In [25]:
prod2_test2_merge = prod2.merge(test2, on='full_table_name', how='inner')
prod2_test2_merge.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


In [26]:
#Эти таблицы есть и на Прод и на Тест. Остальные будем удалять.
not_to_delete = list(prod2_test2_merge.full_table_name.value_counts().index)

In [27]:
not_to_delete[:10] #Первые 10 элементов

['dwh_dds.ref_sc_issue',
 'dwh_dds.ref_deal_forex',
 'dwh_dds.ref_sc_lot',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_account',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.acc_sc_quote',
 'dwh_dds.ref_gen_agreement',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_identcode']

Теперь удаляем строки в Прод и Тест, не входящие в список по полю full_table_name

Прод

In [28]:
indices = prod[~(prod['full_table_name'].isin(not_to_delete))].index
prod.drop(indices, inplace=True)

Тест

In [29]:
indices = test[~(test['full_table_name'].isin(not_to_delete))].index
test.drop(indices, inplace=True)

### Теперь наборы данных Прод и Тест содержат одинаковые таблицы и готовы к сравнению инднксов

In [30]:
len(prod)

573

In [31]:
len(test)

566

Сравнение таблиц по количеству индексов:

In [32]:
tabs_different_number_of_indices = set()

In [33]:
def check_indexes_quantity(row):
    '''Сравнивает количество индексов для таблиц на Прод и Тест и
       если оно различно, добавляет таблицу во множество tabs_different_number_of_indices '''
    if row['inds_count'] != test[test['full_table_name'] == row['full_table_name']].inds_count.max():
        tabs_different_number_of_indices.add(row['full_table_name'])

In [34]:
prod.apply(lambda r: check_indexes_quantity(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7955    None
7956    None
7957    None
7958    None
7959    None
Length: 573, dtype: object

### Вот два списка таблиц, индексы которых будем сверять вручную:

In [35]:
list(tabs_different_number_of_indices)

['dwh_stage.s01_ansignval',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_operjrn',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_deal_scissue',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.acc_doc_payment',
 'dwh_dds.ref_anl_account']

In [36]:
tabs_nonstandard_indexdef

['dwh_stage.s01_led_acc_det_bkp', 'dwh_stage.s01_ledacc_det']

Теперь удалим строки с этими таблицами из Прод и Тест. Остальное можно попробовать сверить в автоматическом режиме

In [37]:
#Общий список таблиц для сверки вручную
tabs_to_manually_comparison = list(tabs_different_number_of_indices) + tabs_nonstandard_indexdef
tabs_to_manually_comparison

['dwh_stage.s01_ansignval',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_operjrn',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_deal_scissue',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.acc_doc_payment',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_led_acc_det_bkp',
 'dwh_stage.s01_ledacc_det']

In [38]:
#Удаляем строки с этими таблицами:
indices = prod[prod['full_table_name'].isin(tabs_to_manually_comparison)].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(tabs_to_manually_comparison)].index
test.drop(indices, inplace=True)

In [39]:
len(prod), len(test)

(498, 498)

**Видим, что на Прод и Тест осталось одинаковое количество строк!**

Сначала попробкем сверить уникальные индексы (primary keys)

Определим таблицы с более чем одним уникальным индексом на Проде и Тесте отдельно:

In [40]:
prod[prod.uniq==1].full_table_name.value_counts()

dwh_stage.s01_t_pcndsc_std            2
dwh_stage.s01_t_acc                   2
dwh_stage.s01_t_dea                   1
dwh_dds.dict_fatca                    1
dwh_stage.s01_l_mrtapprais            1
                                     ..
dwh_stage.s02_f_i                     1
dwh_stage.s01_e_bnkdea                1
dwh_dds.ref_pledge_object             1
dwh_stage.s02_report_type             1
dwh_stage.s02_acnt_balance_history    1
Name: full_table_name, Length: 297, dtype: int64

In [41]:
test[test.uniq==1].full_table_name.value_counts()

dwh_stage.s01_t_pcndsc_std            2
dwh_stage.s01_t_acc                   2
dwh_stage.s01_t_dea                   1
dwh_stage.s01_c_attr_lng              1
dwh_dds.dict_fatca                    1
                                     ..
dwh_dds.ref_product                   1
dwh_stage.s02_f_i                     1
dwh_stage.s01_e_bnkdea                1
dwh_dds.ref_pledge_object             1
dwh_stage.s02_acnt_balance_history    1
Name: full_table_name, Length: 296, dtype: int64

**На Прод и Тест таблицы с более чем одним уникальным индексом одни и те же, НО!, группировка по именам таблиц показывает,
что на Тесте для какой-то таблицы нет уникального индекса при его налияии на Проде. Разные количества строк: 297 на Прод и 296 на Тест**

In [42]:
tabs_diffrent_count_of_unique_indexes = ['dwh_stage.s01_t_acc', 'dwh_stage.s01_t_pcndsc_std']

Добавляем этот список к списку для сверки вручную:

In [43]:
tabs_to_manually_comparison += tabs_diffrent_count_of_unique_indexes
tabs_to_manually_comparison

['dwh_stage.s01_ansignval',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_issue',
 'dwh_stage.s01_t_operjrn',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_deal_scissue',
 'dwh_dm.dm_transaction',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.acc_doc_payment',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_led_acc_det_bkp',
 'dwh_stage.s01_ledacc_det',
 'dwh_stage.s01_t_acc',
 'dwh_stage.s01_t_pcndsc_std']

In [44]:
#Имеем 17 таблиц для сверки индексов вручную:
len(tabs_to_manually_comparison)

17

In [45]:
#Удаляем строки с этими таблицами:
indices = prod[prod['full_table_name'].isin(tabs_diffrent_count_of_unique_indexes)].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(tabs_diffrent_count_of_unique_indexes)].index
test.drop(indices, inplace=True)

In [46]:
len(prod), len(test)

(494, 494)

In [47]:
tabs_diffrent_unique_indexes = set()

In [48]:
def extract_nonunique_indexes_test(row):
    '''Определяет таблицы с неуникальными индексами на Тесте при их уникальности на Проде'''
    
    ind_type_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].idx_type
    if len(ind_type_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_test) < 1:
        print('На Тесте у таблицы ' + row.full_table_name + ' нет уникальных индексов (на Проде есть)') 
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [49]:
prod[prod.uniq==1].apply(lambda r: extract_nonunique_indexes_test(r), axis=1)

На Тесте у таблицы dwh_stage.s01_p_csrjrn нет уникальных индексов (на Проде есть)


1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 295, dtype: object

In [50]:
def extract_nonunique_indexes_prod(row):
    '''Определяет таблицы с неуникальными индексами на Проде при их уникальности на Тесте'''
    
    ind_type_on_prod = prod[(prod.full_table_name==row.full_table_name) & (prod.uniq==1)].idx_type
    if len(ind_type_on_prod) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_prod) < 1:
        print('На Проде у таблицы ' + row.full_table_name + ' нет уникальных индексов (на Тесте есть)') 
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [51]:
test[test.uniq==1].apply(lambda r: extract_nonunique_indexes_prod(r), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
11527    None
11528    None
11530    None
11531    None
11540    None
Length: 294, dtype: object

Таблицы с различием по уникальности индексов на Прод и Тест:

In [52]:
list(tabs_diffrent_unique_indexes)

['dwh_stage.s01_p_csrjrn']

Удалим эту таблицу из рассмотрения на Прод и Тест:

In [53]:
indices = prod[prod['full_table_name'].isin(list(tabs_diffrent_unique_indexes))].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(list(tabs_diffrent_unique_indexes))].index
test.drop(indices, inplace=True)

In [54]:
len(prod), len(test)

(493, 493)

Теперь только можно попробовать сверить уникальные индексы по типам!

In [55]:
#Оставшиеся для анализа (количество) уникальные индексы на Прод и Тест
len(prod[prod.uniq==1]), len(test[test.uniq==1])

(294, 294)

In [56]:
def compare_unique_indexes_types(row):
    '''Сверяет уникальные индексы по типу'''
    #Сначала сверяем по типу:
    ind_type_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].idx_type
    
    if len(ind_type_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_test) < 1:
        raise ValueError('Table ' + row.full_table_name + ' has less then one unuque index')
    
    if ind_type_on_test.iloc[0] != row.idx_type:
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [57]:
prod[prod.uniq==1].apply(lambda r: compare_unique_indexes_types(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 294, dtype: object

In [58]:
tabs_diffrent_unique_indexes

{'dwh_stage.s01_p_csrjrn'}

Никакие таблицы в список не добавились. Значит типы уникальных индексов на Прод и Тест совпадают

Теперь пробуем сверять уникальные индексы по составу полей

In [59]:
def compare_unique_indexes_columns(row):
    '''Сверяет уникальные индексы по полям'''
    #Сначала сверяем по типу:
    ind_columns_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].ind_fields
    
    if len(ind_columns_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_columns_on_test) < 1:
        raise ValueError('Table ' + row.full_table_name + ' has less then one unuque index')
    
    if ind_columns_on_test.iloc[0] != row.ind_fields:
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [60]:
prod[prod.uniq==1].apply(lambda r: compare_unique_indexes_columns(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 294, dtype: object

In [61]:
tabs_diffrent_unique_indexes

{'dwh_stage.s01_p_csrjrn'}

Отлично! Поля в определениях уникальных индексов на Прод и Тест одни и те же!

### Теперь уберем строки с уникальными инднксами и рассмотрим только обычные индексы

In [62]:
indices = prod[prod['uniq']==1].index
prod.drop(indices, inplace=True)

indices = test[test['uniq']==1].index
test.drop(indices, inplace=True)

In [63]:
len(prod), len(test)

(199, 199)